# 鉅亨網的新聞爬蟲--與台股的資料分析
* 按照自由時報那篇格式處理
* [助教的程式碼](https://github.com/MiccWan/Political-News-Analysis/blob/master/crawler/new_talk_crawler.ipynb)

In [1]:
from selenium import webdriver
import numpy as np
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from pathlib import Path, PureWindowsPath
from selenium.webdriver.chrome.options import Options

import sys
import pickle
import requests
from datetime import datetime



# 參考用的
def process_document(document, date):
    
   
    nodes = document.select('ul.list > li')
    #nodes = document.find_all(class_="_2bF theme-list")
    
    data = list()

    for li in nodes:
        # check if is empty element
        if li.select_one('a') == None:
            continue

        # get link
        li_link = 'http://news.ltn.com.tw/' + li.select_one('a')['href']
        #li_link = 'https://news.cnyes.com' + li.select_one('a')['href']
        

        # request for document
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')

        # get date
        li_date = datetime.strptime(date, "%Y%m%d").strftime('%Y-%m-%d')

        #get title
        li_title = li.select_one('p').get_text()

        #get content
        li_content = ""
        for ele in li_doc.select('div.text > p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()

        # append new row
        data.append({
            'date' : li_date,
            'title': li_title,
            'link' : li_link,
            'content' : li_content,
            'tags' : []
        })
    return data

In [2]:
from selenium import webdriver
from time import sleep

url = "https://news.cnyes.com/news/cat/tw_stock"
path = "./googledriver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=path)

driver.get(url)

sleep(2)

js = "document.documentElement.scrollTop = 50000"

driver.execute_script(js)


In [3]:
#res = requests.get('https://news.cnyes.com/news/cat/tw_stock_news')
#doc = bs(res.text, 'lxml')
doc = bs(driver.page_source, 'lxml')

In [4]:
#nodes = doc.find_all(class_="_2bF theme-list")
nodes = doc.find_all(class_="_1Zd")
nodes[0].get('href')

'/news/id/4300859?exp=a'

In [6]:
data = list()
i = 0
for li in nodes:
        # get link
        li_link = 'https://news.cnyes.com' + li.get('href')
        
        # request for document 由各個網站中在抓取該新聞的資料
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')
        
        li_title = li.get('title')
        
        li_content = ""
        for ele in li_doc.find_all('p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()
        
        li_date = li.time.get('datetime')[0:10]
        
        data.append({
            'date' : li_date,
            'link' : li_link,
            'title': li_title,
            'content' : li_content,
        })
        print("第%d則好了"%(i),end = '')
        i = i+1
data

第0則好了第1則好了第2則好了第3則好了第4則好了第5則好了第6則好了第7則好了第8則好了第9則好了第10則好了第11則好了第12則好了第13則好了第14則好了第15則好了第16則好了第17則好了第18則好了第19則好了第20則好了第21則好了第22則好了第23則好了第24則好了第25則好了第26則好了第27則好了第28則好了第29則好了第30則好了第31則好了第32則好了第33則好了第34則好了第35則好了第36則好了第37則好了第38則好了第39則好了第40則好了第41則好了第42則好了

[{'date': '2019-04-10',
  'link': 'https://news.cnyes.com/news/id/4300859?exp=a',
  'title': '神基Q2營收估季增 下半年可望優於上半年',
  'content': '強固型工業電腦廠神基 (3005-TW) 第 1 季受惠強固型電腦出貨穩健、車用、航太營收持穩成長，第 1 季營收略增 2.4%，在公共標案、工業應用帶動下，第 2 季營收估將季增、下半年可望優於上半年。神基 3 月營收 19.37 億元，月增 14.9%、年減 4.7%，第 1 季營收 56.84 億元，季減 12.2%、年增 2.4%。神基表示，第 1 季主要是受惠強固型電腦出貨穩健、車用機構件成長，加上旗下航太扣件廠豐達科 (3004-TW) 首季營收大幅成長挹注下，均帶動營收成長。以神基第 1 季 3 大產品線來看，目前強固型電腦佔 46%，汽車以及筆電機構件佔 43%、航太佔 11%，其中豐達科第 1 季營收 6.35 億元創下歷史次高，年增 29.7%。展望全年，神基認為，強固型電腦除了政府公共標案持穩成長外，工業 4.0 也帶動工業應用需求升溫，加上車用領域開花結果，對第 2 季展望樂觀，可望季增，下半年則可望優於上半年。'},
 {'date': '2019-04-10',
  'link': 'https://news.cnyes.com/news/id/4300835?exp=a',
  'title': '期交所前董事涉違法兼任中國上投摩根獨董 金管會查核被轟有疏漏',
  'content': '立法委員黃國昌今 (10) 日指控，台灣大學財金系教授李存修去 (2018) 年才卸任期交所董事，但他從 2011 年起擔任中國「上投摩根基金管理」獨立董事，不但涉嫌違法兼職，且在擔任期交所董事背景查核過程疑有疏漏。立法院財委會今日赴期交所考察，黃國昌質詢時指控，李存修自 2011 年 8 月就被中國「上投摩根基金管理公司」列為獨立董事，且長年報告都未除名，上投摩根可一路追溯至上海市國有資產兼督管理委員會旗下的國企，且陸委會、教育部皆已認定李存修違法，質疑李存修擔任期交所董事背景查核過程疑有疏漏。對此，期交所總經理黃炳鈞坦言，當時的確沒有蒐集到相關資訊。金管會證期局局長王詠心也表示，李存修的派

In [ ]:
driver.close()

In [ ]:
<a class="_1Zd" title="〈Q2產業景氣展望〉華為手機出貨旺 散熱、砷化鎵族群Q2營運添動能" href="/news/id/4298273" style="display: block;"><div class="_67t theme-meta"><time datetime="2019-04-07T16:30:06+08:00">16:30</time><div class="_jFb theme-sub-cat">台股新聞</div></div><div class="_1xc"><h3>〈Q2產業景氣展望〉華為手機出貨旺 散熱、砷化鎵族群Q2營運添動能</h3><figure class=""><img src="https://cimg.cnyes.cool/prod/news/4298273/xs/b6168fe977414bb74f79345108bae357.jpg" srcset="https://cimg.cnyes.cool/prod/news/4298273/xs/b6168fe977414bb74f79345108bae357.jpg 1x,https://cimg.cnyes.cool/prod/news/4298273/s/b6168fe977414bb74f79345108bae357.jpg 2x,https://cimg.cnyes.cool/prod/news/4298273/m/b6168fe977414bb74f79345108bae357.jpg 3x" alt=""><!-- react-text: 2587 --><!-- /react-text --></figure></div></a>